# Load Data

In [312]:
%matplotlib inline

#basic packages
import pandas as pd
import numpy as np

#viz
import matplotlib.pyplot as plt
import seaborn as sns

#disable warnings
import warnings
warnings.filterwarnings('ignore')

In [329]:
#load data
df_raw = pd.read_csv("yelp_data_comma_delimited.csv", encoding = 'latin1')

In [311]:
# df_raw.head(5)

In [323]:
#print some general statistics
print ("Number of records: ", len(df_raw))
print ("Number of unique users: ", len(df_raw['User - Id'].unique()))
print ("Number of unique retaurants: ", len(df_raw['Business - Id'].unique()))

Number of records:  285764
Number of unique users:  71089
Number of unique retaurants:  5696


# Extract General Category

Yelp provides restaurant options to select multiple categories for each restaurant. Although it's convinient for customers to filter, the overwhelming number of different combinations makes it hard to compare metrics for different categories. 

This section is to compute general categroy labels for each restaurant that best summarizes each restaurants' choice of category combination. For example:

"Sushi Bars, Asian, Restaurants" will be labeled as "Sushi Bars"


In [93]:
print ("Number of unique category: ", len(df_raw['Business - Categories'].unique()))

Number of unique category:  1046


In [266]:
#look at different labels and their review counts
rating_by_cat = df_raw.groupby(df_raw['Business - Categories'])['Review - Stars'].count().reset_index()
rating_by_cat.sort_values('Review - Stars', ascending = False).head(10)

,Business - Categories,Review - Stars
794,"Mexican,Restaurants",25856
23,"American (New),Restaurants",8941
397,"Chinese,Restaurants",7975
853,"Pizza,Restaurants",7165
879,"Restaurants,Italian",7037
51,"American (Traditional),Restaurants",6767
752,"Italian,Pizza,Restaurants",5327
1005,"Thai,Restaurants",5279
983,"Sushi Bars,Japanese,Restaurants",5239
299,"Breakfast & Brunch,Restaurants",4475


In [97]:
#make a list of all unique string values for categories
#make a dictionary of these string values and their counts 

from collections import defaultdict

cat_dict = defaultdict(int)
cat_list = []

def add_to_list (s):
    for cat in s.split(','):
        cat_dict[cat.lower()] += 1
        if cat.lower() not in cat_list:
            cat_list.append(cat.lower())

for cat in df_raw['Business - Categories'].unique():
    add_to_list(cat)


In [100]:
print ("Number of unique category label: ", len(cat_dict))

Number of unique category label:  190


In [101]:
#define function to compute general category refering to the label with the maximum count
def cat_generalize(s):
    #deprioritize general categories
    for cat in ['restaurants', 'bars', 'food', 'nightlife', 'diners', 
                'active Life', 'health & medical', 'food trucks', 'education',
               'arts & entertainment', 'automotive', 'colleges & universities']:
        cat_dict[cat] = -1

    index_max = np.array([cat_dict[cat] for cat in s.lower().split(',')]).argmax()
    return s.split(',')[index_max]

In [104]:
#number of unique general categories
len(df_raw['Business - Categories'].apply(cat_generalize).unique())

86

In [197]:
#unique general categories
df_raw['Business - Categories'].apply(cat_generalize).unique()

array(['Chinese', 'Sandwiches', 'Steakhouses', 'Pizza', 'Mexican',
       'Seafood', 'American (New)', 'Mongolian', 'American (Traditional)',
       'Breakfast & Brunch', 'Wine Bars', 'Burgers', 'Mediterranean',
       'Italian', 'Tex-Mex', 'Lounges', 'Sushi Bars', 'French', 'Thai',
       'Barbeque', 'Buffets', 'Japanese', 'Pubs', 'Restaurants', 'Delis',
       'Hot Dogs', 'Vietnamese', 'Fast Food', 'Vegetarian',
       'Latin American', 'Indian', 'Middle Eastern', 'Asian Fusion',
       'Bars', 'Salad', 'German', 'Spanish', 'Southern',
       'Event Planning & Services', 'Cafes', 'Diners', 'Active Life',
       'Hawaiian', 'Nightlife', 'Greek', 'Specialty Food',
       'Chicken Wings', 'Polish', 'Korean', 'Sports Bars', 'Vegan',
       'Shopping', 'British', 'Turkish', 'Food', 'Bakeries',
       'Gluten-Free', 'Russian', 'Afghan', 'Caribbean', 'Cajun/Creole',
       'Scandinavian', 'Coffee & Tea', 'Taiwanese', 'Grocery',
       'Fish & Chips', 'Irish', 'Filipino', 'Comfort Food', 'Pe

In [314]:
#make a new column for the general label
df_raw['Business - General Category'] = df_raw['Business - Categories'].apply(cat_generalize)

In [108]:
#save dataframe to a new csv for Tableau
df_raw.to_csv("yelp_data_v2.csv")

# Identify the Growth Rate for General Categories

In [199]:
#create columns to store the review date in datatime and year
df_raw['Reveiw - Datetime'] = pd.to_datetime(df_raw['Review - Date'])
df_raw['Review - Year'] = df_raw['Reveiw - Datetime'].map(lambda x: x.year)

In [319]:
#unique years
df_raw['Review - Year'].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014])

In [322]:
print ('Min date: ', df_raw['Reveiw - Datetime'].min())
print ('Max date: ', df_raw['Reveiw - Datetime'].max())

Min date:  2005-02-01 00:00:00
Max date:  2014-07-16 00:00:00


In [318]:
#filtered list of categories with at least 20 restaurants in 2010
cat_review_count = df_raw[df_raw['Review - Year'] == 2010].groupby('Business - General Category')['Business - Id'].nunique().reset_index()
filtered_cat = list(cat_review_count[cat_review_count['Business - Id'] > 20]['Business - General Category'])

#remove generic categories which provides little insights
for cat in filtered_cat:
    if cat.lower() in ['restaurants', 'bars', 'food', 'nightlife', 'diners']:
        filtered_cat.remove(cat)
        
len(filtered_cat)

21

In [282]:
#subset data by year for joins
category_ratings_2010 = df_raw[df_raw['Review - Year'] == 2010].groupby(['Review - Year', 'Business - General Category'])['Review - Stars'].mean().reset_index()
category_ratings_2011 = df_raw[df_raw['Review - Year'] == 2011].groupby(['Review - Year', 'Business - General Category'])['Review - Stars'].mean().reset_index()
category_ratings_2012 = df_raw[df_raw['Review - Year'] == 2012].groupby(['Review - Year', 'Business - General Category'])['Review - Stars'].mean().reset_index()
category_ratings_2013 = df_raw[df_raw['Review - Year'] == 2013].groupby(['Review - Year', 'Business - General Category'])['Review - Stars'].mean().reset_index()

num_2010 = df_raw[df_raw['Review - Year'] == 2010].groupby(['Review - Year', 'Business - General Category'])['Business - Id'].nunique().reset_index()
num_2011 = df_raw[df_raw['Review - Year'] == 2011].groupby(['Review - Year', 'Business - General Category'])['Business - Id'].nunique().reset_index()
num_2012 = df_raw[df_raw['Review - Year'] == 2012].groupby(['Review - Year', 'Business - General Category'])['Business - Id'].nunique().reset_index()
num_2013 = df_raw[df_raw['Review - Year'] == 2013].groupby(['Review - Year', 'Business - General Category'])['Business - Id'].nunique().reset_index()

In [289]:
#join data for YoY growth rate comparison
df1 = pd.merge(category_ratings_2010[['Business - General Category', 'Review - Stars']], 
             category_ratings_2011[['Business - General Category', 'Review - Stars']],
             on = 'Business - General Category', suffixes = ['_2010', '_2011'], how = 'left')

df2 = pd.merge(df1, category_ratings_2012[['Business - General Category', 'Review - Stars']],
             on = 'Business - General Category', suffixes = ['_2011', '_2012'], how = 'left')

df3 = pd.merge(df2, category_ratings_2013[['Business - General Category', 'Review - Stars']],
             on = 'Business - General Category', suffixes = ['_2012', '_2013'], how = 'left')

#merge number of restaurants
df4 = pd.merge(df3, num_2010[['Business - General Category', 'Business - Id']],
             on = 'Business - General Category', suffixes = ['', '_2010'], how = 'left')

df5 = pd.merge(df4, num_2011[['Business - General Category', 'Business - Id']],
             on = 'Business - General Category', suffixes = ['_2010', '_2011'], how = 'left')

df6 = pd.merge(df5, num_2012[['Business - General Category', 'Business - Id']],
             on = 'Business - General Category', suffixes = ['_2011', '_2012'], how = 'left')

df7 = pd.merge(df6, num_2013[['Business - General Category', 'Business - Id']],
             on = 'Business - General Category', suffixes = ['_2012', '_2013'], how = 'left')

In [305]:
#filter to the categories with at least 30 review in 2010
df8 = df7[df7['Business - General Category'].isin(filtered_cat)]
df8.columns = ['category', 'stars_10',
       'stars_11', 'stars_12', 'stars_13',
       'num_10', 'num_11', 'num_12',
       'num_13']
df8

,category,stars_10,stars_11,stars_12,stars_13,num_10,num_11,num_12,num_13
2,American (New),3.856855,3.865489,3.873238,3.853502,214,258.0,316.0,396.0
3,American (Traditional),3.550792,3.562487,3.539097,3.550908,410,463.0,531.0,594.0
6,Barbeque,3.598624,3.742215,3.683374,3.699687,34,37.0,43.0,53.0
7,Breakfast & Brunch,3.799099,3.816955,3.818927,3.848981,112,135.0,150.0,177.0
9,Buffets,3.782895,3.759637,3.771134,3.611702,35,39.0,42.0,52.0
10,Burgers,3.665129,3.628337,3.580709,3.571891,112,182.0,208.0,259.0
11,Cafes,3.992593,3.802752,3.963211,3.944598,22,29.0,47.0,67.0
15,Chinese,3.613099,3.603526,3.552932,3.622719,258,305.0,331.0,360.0
18,Delis,3.773196,3.782946,3.748148,3.693642,25,27.0,29.0,30.0
22,Fast Food,3.516340,3.255882,3.229474,3.088520,87,158.0,193.0,236.0


In [306]:
#compute individual and average growth rate
df8['star_growth_11'] = (df8['stars_11'] - df8['stars_10'])/(df8['stars_10'])
df8['star_growth_12'] = (df8['stars_12'] - df8['stars_11'])/(df8['stars_11'])
df8['star_growth_13'] = (df8['stars_13'] - df8['stars_12'])/(df8['stars_12'])
df8['avg_star_growth'] = (df8['star_growth_11'] + df8['star_growth_12'] 
                             + df8['star_growth_13']) * 100 / 3

df8['num_growth_11'] = (df8['num_11'] - df8['num_10'])/(df8['num_10'])
df8['num_growth_12'] = (df8['num_12'] - df8['num_11'])/(df8['num_11'])
df8['num_growth_13'] = (df8['num_13'] - df8['num_12'])/(df8['num_12'])
df8['avg_num_growth'] = (df8['num_growth_11'] + df8['num_growth_12'] 
                             + df8['num_growth_13']) * 100 / 3

In [307]:
#top 5 categories in terms of rating growth rate
df8[['category', 'avg_star_growth']].sort_values('avg_star_growth', ascending = False).head(5)

,category,avg_star_growth
6,Barbeque,0.953566
64,Sushi Bars,0.676705
42,Mediterranean,0.629215
43,Mexican,0.515287
7,Breakfast & Brunch,0.436211


In [308]:
#bottom 5 categorie in terms of rating growth rate
df8[['category', 'avg_star_growth']].sort_values('avg_star_growth').head(5)

,category,avg_star_growth
22,Fast Food,-4.194257
38,Japanese,-2.068309
9,Buffets,-1.512234
37,Italian,-0.868357
10,Burgers,-0.854254


In [309]:
#top 5 categorie in terms of restaurant growth rate
df8[['category', 'avg_num_growth']].sort_values('avg_num_growth', ascending = False).head(5)

,category,avg_num_growth
11,Cafes,45.480113
22,Fast Food,42.013629
10,Burgers,33.768315
2,American (New),22.785941
55,Sandwiches,22.386724


In [310]:
#bottom 5 categorie in terms of restaurant growth rate
df8[['category', 'avg_num_growth']].sort_values('avg_num_growth').head(5)

,category,avg_num_growth
57,Seafood,6.195120
18,Delis,6.285228
37,Italian,6.783553
63,Steakhouses,7.391874
15,Chinese,11.834325


# Appendix

This was scripted in an effort to summarize the 'general labels'. However, the results weren't as meaningful as I expected. I ended us just using the "popular vote" method to produce the general labels for each restaurant

In [331]:
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *


from sklearn.decomposition import NMF
from collections import Counter 
import re
import string

# text processor
def pre_process(s):
    s = re.sub("[^\w']|_", " ", s) 
    s=s.translate(str.maketrans(' ',' ',string.punctuation))# Strip punctuation before looking
    s= re.sub(' +',' ', s) # Remove extra spaces
    s=s.lower()
    return s

no_features = 100
no_topics = 40
no_top_words = 2

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, preprocessor = pre_process, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df_raw['Business - Categories'].unique())
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Print most common topics
topics_vec = np.argmax(nmf.transform(tfidf),axis=1)
imp_topics = Counter(topics_vec).most_common(5)
print('Top five most common topics with counts: ' + str(imp_topics))

# Derived topics with the top words in each topic
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

print("NMF Topics:\n")
display_topics(nmf, tfidf_feature_names, no_top_words)

Top five most common topics with counts: [(0, 86), (1, 47), (5, 42), (10, 36), (3, 36)]
NMF Topics:

Topic 0:
bars nightlife
Topic 1:
food specialty
Topic 2:
new american
Topic 3:
breakfast brunch
Topic 4:
pizza italian
Topic 5:
asian fusion
Topic 6:
mediterranean hotels
Topic 7:
traditional american
Topic 8:
sandwiches cheesesteaks
Topic 9:
hot dogs
Topic 10:
steakhouses seafood
Topic 11:
mexican soup
Topic 12:
barbeque hawaiian
Topic 13:
tea coffee
Topic 14:
event services
Topic 15:
wings chicken
Topic 16:
gluten free
Topic 17:
cafes caribbean
Topic 18:
arts entertainment
Topic 19:
ice cream
Topic 20:
burgers sports
Topic 21:
buffets thai
Topic 22:
seafood chips
Topic 23:
japanese sushi
Topic 24:
fast food
Topic 25:
salad pizza
Topic 26:
chinese thai
Topic 27:
greek southern
Topic 28:
delis bagels
Topic 29:
eastern middle
Topic 30:
bakeries food
Topic 31:
vegetarian vegan
Topic 32:
wine spirits
Topic 33:
diners lounges
Topic 34:
vietnamese thai
Topic 35:
juice smoothies
Topic 36:
des